In [20]:
import tensorflow as tf
import pandas as pd
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

In [50]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

X = train_df.drop('target', axis=1) 
y = train_df['target']               # Target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='error')

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'lambda': [0, 0.1]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=4, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)



In [ ]:
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)

val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {val_accuracy:.4f}')
print('Validation Classification Report:')
print(classification_report(y_val, y_val_pred))
print('Validation Confusion Matrix:')
print(confusion_matrix(y_val, y_val_pred))

X_test = test_df.drop('target', axis=1, errors='ignore')
X_test = X_test.reindex(columns=X_train.columns, fill_value=0) 

y_test_pred = best_model.predict(X_test)
np.savetxt('predictions.txt', y_test_pred, fmt='%d', delimiter='')

print('Predictions have been saved to predictions.txt.')
